<a href="https://colab.research.google.com/github/h0806449f/PyTorch/blob/main/NLP_first_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **==  0. HuggingFace ==**

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

from transformers import pipeline

In [5]:
# 情緒分析
classifier = pipeline(model = "distilbert-base-uncased-finetuned-sst-2-english", # Dfault model
                      task = "sentiment-analysis")


classifier("首次嘗試使用NLP相關模型, 模型來自於HuggingFace, 看起來有點厲害")

[{'label': 'NEGATIVE', 'score': 0.970554769039154}]

In [11]:
# 零樣本 - 文本分類
classifier = pipeline(model = "facebook/bart-large-mnli", # Default model
                      task = "zero-shot-classification")

classifier("This is a course about the Transformers library",
           candidate_labels=["education", "politics", "business"])

{'sequence': 'This is a course about the Transformers library',
 'labels': ['education', 'business', 'politics'],
 'scores': [0.8445989489555359, 0.11197412759065628, 0.04342695698142052]}

In [12]:
# 文本生成
generator = pipeline(model = "gpt2", # Default model
                     task = "text-generation")

generator("These are some steps for build risk forecast model",
          max_new_tokens = 50)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'These are some steps for build risk forecast model, such as using data with different forecast values.'}]

In [14]:
# 文本生成
generator = pipeline("text-generation", model="distilgpt2")

generator(
    "These are some steps for build risk forecast model",
    max_length=30,
    num_return_sequences=2,
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'These are some steps for build risk forecast model development with tools that can be useful and use cases for understanding how to use our current software to evaluate your'},
 {'generated_text': 'These are some steps for build risk forecast model (see below).\n\n\n\n\nPrerequisites\nLet’s begin in a short time'}]